In [1]:
import numpy as np
import h5py
import sys
import logging
sys.path.append('../')

# Neural network stuff
from fielutil import load_verbatimnet
from featextractor import extract_imfeats_debug
from data_iters.hdf5_iterator import Hdf5MiniBatcher
from data_iters.minibatcher import MiniBatcher
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical


Using Theano backend.


Using gpu device 0: GeForce GTX 980


In [2]:
# Do you want to load the features in? Or save them to a file?
load_features = False

# All the images that you require extraction should be in this HDF5 file
hdf5authors='nmecdata/nmec_scaled_author_form.hdf5'
hdf5images='nmecdata/nmec_scaled_flat.hdf5'

# This is the file that you will load the features from or save the features to
# featurefile = 'icdar13data/benchmarking-processed/icdar13be_fiel657.npy'
# featurefile = 'icdar13data/experimental-processed/icdar13ex_fiel657.npy'
featurefile = 'nmecdata/nmec_fiel657_features.npy'

# This is the neural networks and parameters you are deciding to use
paramsfile = '/fileserver/iam/iam-processed/models/fiel_657.hdf5'

In [3]:
labels = h5py.File(hdf5authors)
num_authors=len(labels)
num_forms_per_author=-1
shingle_dim=(56,56)
batch_size=32
iterations = 1000
lr = 0.001

In [4]:
vnet = load_verbatimnet( 'fc7', paramsfile=paramsfile, compiling=False )
vnet.add(Dense(num_authors))
vnet.add(Activation('softmax'))
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
vnet.compile(loss='categorical_crossentropy', optimizer=sgd)

Establishing Fiel's verbatim network
Loaded neural network up to fc7 layer


In [5]:
nmec_m = Hdf5MiniBatcher(hdf5authors, num_authors, num_forms_per_author,
                            shingle_dim=shingle_dim, default_mode=MiniBatcher.TRAIN,
                            batch_size=batch_size)

In [6]:
for i in range(iterations):
    (X_train,Y_train) = nmec_m.get_train_batch()
    X_train = 1.0 - X_train / 255.0
    X_train = np.expand_dims(X_train, 1)
    Y_train = to_categorical(Y_train, num_authors)
    vnet.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1, show_accuracy=True, verbose=1)
    print "Finished training on the "+str(i)+"th batch"
    if (batch_iter % 20)==0 and batch_iter != 0:
        model.save_weights('fielnet-1k.hdf5', overwrite=True)

Epoch 1/1


RuntimeError: BaseGpuCorrMM: Failed to allocate output of 96 x 96 x -3 x -3
Apply node that caused the error: GpuCorrMM_gradWeights{valid, (1, 1)}(GpuContiguous.0, GpuContiguous.0)
Toposort index: 318
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D)]
Inputs shapes: [(32, 96, 10, 10), (32, 96, 14, 14)]
Inputs strides: [(9600, 100, 10, 1), (18816, 196, 14, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuDimShuffle{1,0,2,3}(GpuCorrMM_gradWeights{valid, (1, 1)}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [7]:
X_train.shape

(32, 1, 120, 120)